# Machine Learning on Cell: Random Forest

In [70]:
from sklearn.ensemble import RandomForestClassifier

import pandas as pd

import pandas.io.sql as sqlio

import psycopg2

import matplotlib.pyplot as plt

In [99]:
zoom = 5

In [100]:
conn = psycopg2.connect("dbname=cell user=cell_master password=38fn3k39erj34n8 host=kepler port=5632")
sql = "select * from ml.urbanization_training_%s" % zoom

In [101]:
data_training = sqlio.read_sql_query(sql, conn)
data_training.head()

,zoom,x,y,r0_18,r18_64,r64,residencial,total_superficie_cons,gid_tz,class,class_descriptor
0,5,398,475,0.36,0.60,0.04,71.0,14506.0,3,3,Tomares - Urbanizaciones
1,5,398,476,0.33,0.63,0.04,569.0,94246.0,3,3,Tomares - Urbanizaciones
2,5,398,477,0.28,0.61,0.10,24.0,7672.0,3,3,Tomares - Urbanizaciones
3,5,399,475,0.33,0.62,0.04,473.0,103205.0,3,3,Tomares - Urbanizaciones
4,5,399,476,0.21,0.72,0.07,873.0,117305.0,3,3,Tomares - Urbanizaciones


In [102]:
data_training.describe()

,zoom,x,y,r0_18,r18_64,r64,residencial,total_superficie_cons,gid_tz,class
count,59.0,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000,59.000000
mean,5.0,406.457627,476.220339,0.166780,0.645085,0.187966,1612.288136,301438.203390,1.338983,1.338983
std,0.0,4.530822,2.901057,0.063258,0.040530,0.078779,1115.585806,194251.536945,1.397167,1.397167
min,5.0,398.000000,467.000000,0.090000,0.580000,0.010000,24.000000,7672.000000,0.000000,0.000000
25%,5.0,401.500000,475.000000,0.130000,0.610000,0.150000,708.000000,125722.000000,0.000000,0.000000
50%,5.0,408.000000,476.000000,0.150000,0.640000,0.210000,1538.000000,267886.000000,1.000000,1.000000
75%,5.0,410.000000,478.000000,0.175000,0.670000,0.240000,2614.000000,458236.000000,3.000000,3.000000
max,5.0,412.000000,481.000000,0.360000,0.720000,0.330000,4151.000000,781754.000000,4.000000,4.000000


In [103]:
clases = data_training["class"]
clases.head()

0    3
1    3
2    3
3    3
4    3
Name: class, dtype: int64

In [104]:
vectors_training = data_training[[ "r0_18", "r18_64", "r64", "residencial", "total_superficie_cons" ]]
vectors_training.head()

,r0_18,r18_64,r64,residencial,total_superficie_cons
0,0.36,0.60,0.04,71.0,14506.0
1,0.33,0.63,0.04,569.0,94246.0
2,0.28,0.61,0.10,24.0,7672.0
3,0.33,0.62,0.04,473.0,103205.0
4,0.21,0.72,0.07,873.0,117305.0


In [105]:
rf = RandomForestClassifier(n_estimators=50).fit(vectors_training, clases)

In [106]:
sql = "select * from ml.urbanization_%s" % zoom
data = sqlio.read_sql_query(sql, conn)
data.head()

,zoom,x,y,r0_18,r18_64,r64,residencial,total_superficie_cons,geom
0,5,156,491,0.18,0.68,0.14,172.0,21936.0,0103000020DB0B0000010000000500000000000000C831...
1,5,156,492,0.15,0.68,0.17,188.0,36927.0,0103000020DB0B0000010000000500000000000000C831...
2,5,156,493,0.16,0.69,0.15,137.0,49061.0,0103000020DB0B0000010000000500000000000000C831...
3,5,156,494,0.11,0.67,0.21,851.0,155615.0,0103000020DB0B0000010000000500000000000000C831...
4,5,156,495,0.16,0.66,0.18,553.0,82854.0,0103000020DB0B0000010000000500000000000000C831...


In [107]:
data["class"] = rf.predict(data[["r0_18", "r18_64", "r64", "residencial", "total_superficie_cons"]])
data[["class", "r0_18", "r18_64", "r64", "residencial", "total_superficie_cons"]].head()

,class,r0_18,r18_64,r64,residencial,total_superficie_cons
0,3,0.18,0.68,0.14,172.0,21936.0
1,0,0.15,0.68,0.17,188.0,36927.0
2,3,0.16,0.69,0.15,137.0,49061.0
3,0,0.11,0.67,0.21,851.0,155615.0
4,0,0.16,0.66,0.18,553.0,82854.0


In [108]:
data[["class", "r0_18", "r18_64", "r64", "residencial", "total_superficie_cons"]].groupby(["class"]).agg([ "count", "mean" ])

r0_18           r18_64             r64           residencial  \
      count      mean  count      mean count      mean       count   
class                                                                
0      2832  0.115710   2832  0.689693  2832  0.194672        2832   
1      1755  0.106034   1755  0.567117  1755  0.327214        1755   
2       341  0.204457    341  0.683578   341  0.112170         341   
3      4881  0.181987   4881  0.730324  4881  0.087837        4881   
4       546  0.181154    546  0.685275   546  0.133700         546   

                   total_superficie_cons                 
              mean                 count           mean  
class                                                    
0       500.612288                  2832   91051.655720  
1       481.615954                  1755   86577.264957  
2      1417.310850                   341  226842.451613  
3       167.053063                  4881   36643.745954  
4       789.939560                   546  140532.675824

In [109]:
cur = conn.cursor()
sql = ""

for i, r in data.iterrows():
    sql = sql + 'update cell_data.data set data = data || \'{ "9932": %i }\'::jsonb where zoom=%i and x=%i and y=%i;' % (r["class"], r["zoom"], r["x"], r["y"])

In [110]:
cur.execute(sql)
conn.commit()

In [111]:
cur.close()

In [112]:
conn.close()